In [315]:
import requests
import json
import pandas as pd
import numpy as np
import os

r = requests.get("https://restcountries.eu/rest/v2/region/europe")
rc = requests.get("https://api.coinbase.com/v2/exchange-rates")

# ramka główna
response_json = r.json()
response_c_json = rc.json()
df = pd.DataFrame(response_json)

# ramka do walut w PLN
dfc = pd.DataFrame.from_dict(response_c_json.get("data"))
dfc.reset_index(inplace=True)

In [316]:
# Konwertuję languages, topLevelDomain, timezones do odpowiedniego formatu

languages = []

for language in df["languages"]:
    language = language[0].get("name")
    languages.append(language)
    
languages = pd.Series(languages)
df["languages"] = languages


topLevelDomain = []

for domain in df["topLevelDomain"]:
    domain = domain[0]
    topLevelDomain.append(domain)

topLevelDomain = pd.Series(topLevelDomain)
df["topLevelDomain"] = topLevelDomain

timezones = []

for timezone in df["timezones"]:
    timezone = " ".join(timezone)
    timezones.append(timezone)

timezones = pd.Series(timezones)
df["timezones"] = timezones


In [317]:
# Tworzę kolumnę typu boolean z informacją czy EU

blocs = []

for ls in df["regionalBlocs"]:
    if ls == []:
        ls = "NA"
    else:
        ls = ls[0].get('acronym')
    blocs.append(ls)
    
blocs = pd.Series(blocs)

is_eu = (blocs == 'EU')
df["is_eu"] = is_eu

In [318]:
# Tworzę kolumnę z dystansem do Polski

from math import cos, asin, sqrt, pi

def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) 

lat_pl, lon_pl = df.loc[df["name"] == 'Poland', 'latlng'].values[0]

dist_to_pl = []
for values in df["latlng"]:
    dist = distance(values[0], values[1], lat_pl, lon_pl)
    dist_to_pl.append(dist)

dist_to_pl = pd.Series(dist_to_pl)
df["dist_to_pl"] = dist_to_pl

In [319]:
# Tworzę kolumny z kodem i nazwą waluty

currency_code = []
currency_name = []

for cr in df["currencies"]:
    cr_code = cr[0].get('code')
    cr_name = cr[0].get('name')
    currency_code.append(cr_code)
    currency_name.append(cr_name)

currency_code = pd.Series(currency_code)
df["currency_code"] = currency_code

currency_name = pd.Series(currency_name)
df["currency_name"] = currency_name

In [320]:
# Tworzę kolumnę w drugiej ramce z wartościami walut w PLN

def currency(c1, c2):
    return c1/c2

pln = dfc.loc[dfc["index"] == 'PLN', 'rates'].values[0]
pln = float(pln)

rate_pln = []

for rate in dfc["rates"]:
    rate = float(rate)
    rate_conv = currency(rate, pln)
    rate_pln.append(rate_conv)

rate_pln = pd.Series(rate_pln)
dfc["rate_pln"] = rate_pln

dfc = dfc[["index", "rate_pln"]]

In [321]:
# Tworzę ramkę finalną

df_final = df[["name", "nativeName", "alpha2Code", "area", "population", "capital", "languages", "topLevelDomain", "timezones","is_eu","dist_to_pl","currency_code", "currency_name"]]

# Dołączam kolumnę z drugiej ramki zawierającą wartości w PLN

df_final = pd.merge(df_final, dfc, how="left", left_on="currency_code", right_on="index")
del df_final["index"] 

df_final.head()

,name,nativeName,alpha2Code,area,population,capital,languages,topLevelDomain,timezones,is_eu,dist_to_pl,currency_code,currency_name,rate_pln
0,Åland Islands,Åland,AX,1580.0,28875,Mariehamn,Swedish,.ax,UTC+02:00,True,902.553273,EUR,Euro,0.221006
1,Albania,Shqipëria,AL,28748.0,2886026,Tirana,Albanian,.al,UTC+01:00,False,1223.144193,ALL,Albanian lek,27.147982
2,Andorra,Andorra,AD,468.0,78014,Andorra la Vella,Catalan,.ad,UTC+01:00,False,1742.525377,EUR,Euro,0.221006
3,Austria,Österreich,AT,83871.0,8725931,Vienna,German,.at,UTC+01:00,True,706.159582,EUR,Euro,0.221006
4,Belarus,Белару́сь,BY,207600.0,9498700,Minsk,Belarusian,.by,UTC+03:00,False,552.515296,BYN,New Belarusian ruble,0.699793


In [322]:
#Eksport pliku

export_filename = "zadanie.csv"
df_final.to_csv(os.path.join(os.path.abspath(""), export_filename))